In [ ]:
%pip install pyautogen

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
import autogen

# Load LLM inference endpoints from an env variable or a file
# See https://microsoft.github.io/autogen/docs/FAQ#set-your-api-endpoints
# and OAI_CONFIG_LIST_sample.json
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")
llm_config = {"config_list": config_list, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin. You only interact with the Coordinator to generate a poem on the provided topic.",
   code_execution_config=False,
   human_input_mode="TERMINATE"
)
coordinator = autogen.AssistantAgent(
    name="Coordinator",
    system_message="You are the Coordinator. You will send tasks to the Poet only. You will always prompt the poet to generate poems about the ORIGINAL TOPIC until the critic deems one 8 or higher. If it is an 8 or higher, respond with TERMINATE",
    llm_config=llm_config,
)
poet = autogen.AssistantAgent(
    name="Poet",
    system_message="You are a poet. You respond with a poem and nothing else. Your poems are always 10 words long. You can talk to Critic only.",
    llm_config=llm_config,
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="You are a critic. You return nothing but a 0-10 score for a poem. To be higher than 8, a poem must be very good. You can talk to Coordinator only.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coordinator, poet, critic], messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
''',
)
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code."""
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
''',
    llm_config=gpt4_config,
)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "paper"},
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt4_config,
)

In [ ]:
coordinator.initiate_chat(manager, message="The topic is Atlantic Ocean.")

In [ ]:
%pip install yfinance

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt

# Define the ticker symbols for the stocks
nvda_ticker = "NVDA"
tesla_ticker = "TSLA"

# Download the historical stock price data
nvda_data = yf.download(nvda_ticker, start="2021-01-01", end="2021-12-31")
tesla_data = yf.download(tesla_ticker, start="2021-01-01", end="2021-12-31")

# Extract the "Close" price from the data
nvda_close = nvda_data["Close"]
tesla_close = tesla_data["Close"]

# Plot the stock price change YTD
plt.plot(nvda_close.index, nvda_close, label="NVDA")
plt.plot(tesla_close.index, tesla_close, label="TSLA")
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.title("NVDA and TESLA Stock Price Change YTD")
plt.legend()
plt.show()

In [ ]:
# Termination message detection
def is_termination_msg(content) -> bool:
    have_content = content.get("content", None) is not None
    if have_content and "TERMINATE" in content["content"]:
        return True
    return False

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
import autogen

# Load LLM inference endpoints from an env variable or a file
# See https://microsoft.github.io/autogen/docs/FAQ#set-your-api-endpoints
# and OAI_CONFIG_LIST_sample.json
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")
llm_config = {"config_list": config_list, "cache_seed": 42}
design_system_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config=False,
   is_termination_msg=is_termination_msg,
   human_input_mode="NEVER"
)
facilitator = autogen.AssistantAgent(
    name="Facilitator",
    system_message="""You are the Facilitator. You gather opinions from EVERYONE (both the Marketer and the Customer Expert), then select the best 3 examples among everything brainstormed, followed by the word TERMINATE. Do not write anything else.""",
    llm_config=llm_config,
)
marketer = autogen.AssistantAgent(
    name="Marketer",
    system_message="""You are the Marketer. You are the CMO of this company who cares about the company's brand identify. Share 3 one-sentence goals based on your area of expertise. Do not write anything else.""",
    llm_config=llm_config,
)
customer_expert = autogen.AssistantAgent(
    name="Customer_expert",
    system_message="""You are the Customer Expert. You always bring in the customer's perspective. Share 3 one-sentence goals based on your area of expertise. Do not write anything else.""",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[design_system_proxy, facilitator, marketer, customer_expert], messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
facilitator.initiate_chat(manager, message="Help retirees transition to a life without work.")

### Brainstorm questions

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
import autogen

# Load LLM inference endpoints from an env variable or a file
# See https://microsoft.github.io/autogen/docs/FAQ#set-your-api-endpoints
# and OAI_CONFIG_LIST_sample.json
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")
llm_config = {"config_list": config_list, "cache_seed": 42}
design_system_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config=False,
   is_termination_msg=is_termination_msg,
   human_input_mode="NEVER"
)
facilitator = autogen.AssistantAgent(
    name="Facilitator",
    system_message="""You are the Facilitator. You gather opinions from EVERYONE (both the Marketer and the Customer Expert) to collect questions about the sprint goal, which is: "Help retirees transition healthily out of work." We assume a lot of things, so we want to look pessimistically and wonder how we might fail to achieve this goal. Each expert should provide 5 questions such as "Will customers trust our expertise?". Once you have collected questions, return the top 3 that you think are most important, followed by the word TERMINATE.""",
    llm_config=llm_config,
)
marketer = autogen.AssistantAgent(
    name="Marketer",
    system_message="""You are the Marketer. You are the CMO of this company who cares about the company's brand.""",
    llm_config=llm_config,
)
customer_expert = autogen.AssistantAgent(
    name="Customer_expert",
    system_message="""You are the Customer Expert. You always bring in the customer's perspective.""",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[design_system_proxy, facilitator, marketer, customer_expert], messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
facilitator.initiate_chat(manager, message="Help retirees transition to a life without work.")